<a href="https://colab.research.google.com/github/aswinaus/graphrag/blob/main/Graph_RAG_GraphDBStorage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyvis IPython cchardet datasets langchain==0.1.17 neo4j openai tiktoken langchain-community langchain-experimental json-repair
from neo4j import GraphDatabase
from google.colab import userdata

url = userdata.get('NEO4J_URI')
username ="neo4j"
password = userdata.get('NEO4J_PASSWORD')

driver = GraphDatabase.driver(url, auth=(username, password))

In [4]:
def add_graph_to_neo4j(driver, graph):
    with driver.session() as session:
        # Add Nodes
        for node, attrs in graph.nodes(data=True):
            cypher_query = """
            MERGE (n:__Entity__ {{name:
entity}})
            SET n += $props
            WITH n
            CALL apoc.create.addLabels( n, ['{label}'] ) YIELD node
            RETURN distinct 'done' AS result
            """.format(label=attrs['entity'])  # Dynamically set the label based on the 'entity' attribute
            session.run(cypher_query, name=node, entity=attrs['entity'], props={k: v for k, v in attrs.items() if k not in ['entity']})

        # Add Edges
        for source, target, attrs in graph.edges(data=True):
            cypher_query = """
            MATCH (a),(b)
            WHERE a.name =
target
            MERGE (a)-[r:{relationship}]->(b)
            SET r += $props
            """.format(relationship=attrs['relationship'])  # Dynamically set the relationship type
            session.run(cypher_query, source=source, target=target, props={k: v for k, v in attrs.items() if k not in ['relationship']})

# Finally, call the function to add your graph to Neo4j
add_graph_to_neo4j(driver, G)


NameError: name 'G' is not defined